In [ ]:
from utils import *
from utils2 import *
from gpcrdb_soup import *

In [ ]:
import Bio
import yaml
import os
from functools import partial
from operator import is_not
from tqdm import tqdm

In [ ]:
uniprot = False  # specify if we want to scrape for uniprot links ~

In [ ]:
# table = get_table(reload=True, save=True, uniprot=uniprot)

In [ ]:
table = get_table(reload=False)

In [ ]:
if uniprot:
    t = table[['PDB', 'pdb_link', 'uniprot(gene)','uniprot_id', 'uniprot_link']]
else:
    t = table[['PDB', 'pdb_link', 'uniprot(gene)']]

In [ ]:
t
# Todo: https://www.ebi.ac.uk/training/online/sites/ebi.ac.uk.training.online/files/UniProt_programmatically_py3.pdf

In [ ]:
df = table

# This is the thing we might want to specify with argparse
filtered = df[df['Cl.'].str.contains('A')]
filtered = filtered[filtered['Species'].str.contains('Human')]

In [ ]:
filtered

In [ ]:
# 2.1) find active state (complex i.p.)
filtered_complex = filtered[filtered['Family'].str.contains('Gs')]
active = filtered_complex[filtered_complex['State'].str.contains('Active')]

In [ ]:
active

In [ ]:
# todo: check if file already exists and use overwrite=True/False
"""
for url in tqdm(active['pdb_link']):
    download_pdb(url, 'active')
"""

In [ ]:
# Data loading
files_a, prots_a = get_pdb_files(path='data/pdb/active/')  # Get all downloaded pdb files in specified path
print("Found {} proteins: {}.".format(len(prots_a), prots_a))

In [ ]:
# find inactive counter parts of the active proteins ==> use gene/uniprot, Family and species to match them

genes = list(set(active['uniprot(gene)'].values.tolist()))
print(genes)

inactive = filtered[(filtered['uniprot(gene)'].isin(genes))
                    & (filtered['State'].str.contains('Inactive')) 
                    & (filtered['Species'].str.contains('Human'))]

In [ ]:
"""for url in tqdm(inactive['pdb_link']):
    download_pdb(url, 'inactive')"""

In [ ]:
inactive

In [ ]:
# Data loading
files_i, prots_i = get_pdb_files(path='data/pdb/inactive/')  # Get all downloaded pdb files in specified path
print("Found {} proteins: {}.".format(len(prots_i), prots_i))

In [ ]:
try:
    seg_aligns = get_alignment(prots_a)
    print(yaml.dump(seg_aligns, default_flow_style=False))
except:
    print("Doesnt work anymore...")

In [ ]:
al_df = load_alignment_()

In [ ]:
al_df = clean_alignment(al_df)

In [ ]:
len(al_df)  # this is the entire class A!

In [ ]:
disp3(al_df)

In [ ]:
# since the identifiers are only human readable...

In [ ]:
genes = list(al_df['clean_id'])

l4=[]
l5=[]
l6=[]

for g in genes:
    if len(g) == 4:
        l4.append(g)
    elif len(g) == 5:
        l5.append(g)
    elif len(g) == 6:
        l6.append(g)
    else:
        pass

In [ ]:
# print(l4)  # these are (all?) pdbs

In [ ]:
# print(l5)  # these are?

In [ ]:
# print(l6)  # another id..

In [ ]:
inactive['pdb_lower'] = inactive.PDB.apply(lambda x: x.lower())
active['pdb_lower'] = active.PDB.apply(lambda x: x.lower())

In [ ]:
found = []
for g in l4:
    select = inactive[inactive['pdb_lower'].str.contains(g.lower())]
    if len(select) > 0:
        found.append(g)

In [ ]:
print("Found {} / {} inactive proteins.".format(len(found), len(inactive)))

In [ ]:
# sorted(list(inactive['PDB'].str.lower()))[:10]

In [ ]:
# sorted(l4)[:10]

In [ ]:
"""for g in l5:
    select = inactive[inactive['uniprot(gene)'].str.contains(g)]
    if len(select) > 0:
        print(select['uniprot(gene)'])"""

In [ ]:
"""for g in l6:
    select = inactive[inactive['uniprot_id'].str.contains(g)]
    print(select['uniprot(gene)'])"""

In [ ]:
# files_a

In [ ]:
structures_a = pdb_data(files_a)

In [ ]:
structures_i = pdb_data(files_i)

In [ ]:
al_df

In [ ]:
disp3(structures_i) # missing segment information, segment position information, pp position information

In [ ]:
full_a = structure_to_full(active, structures_a, al_df)

In [ ]:
full_a.to_csv('full_a.csv')
full_a = pd.read_csv('full_a.csv')

In [ ]:
# full_a['full_prot_seq']

In [ ]:
# full_a['TM7_combined']

In [ ]:
# full_a['H8_combined']

In [ ]:
complete_a = get_align_dict(full_a)

In [ ]:
# Filter out poorly aligned proteins
disp3(complete_a[complete_a['std'] < 20])

In [ ]:
full_i = structure_to_full(inactive, structures_i, al_df)

In [ ]:
full_i.to_csv('full_i.csv')
full_i = pd.read_csv('full_i.csv')

In [ ]:
print(len(full_i))

In [ ]:
complete_i = get_align_dict(full_i)
# Filter out poorly aligned proteins
disp3(complete_i[complete_i['std'] < 20])

In [ ]:
len(complete_i[complete_i['std'] < 10])

In [ ]:
disp3(full_i)

In [ ]:
print(len(complete_i))

In [ ]:
# Fix that number ~ we have 73 proteins in the full datatable but end up with 87 after the segments alignment is complete

In [ ]:
# extract target sequence 

In [ ]:
# match each of the active complexes to all the inactive ones (based on the uniprot genes)

In [ ]:
# Get secondary structure

# given their sequences we do our own sequence  alignment (of the pdb file sequence) to the downloaded sequences (to see which matches best)
# then we add the secondary structure information to the data table
# we number the residues in the pdb sequence (based on their position in the )
# we can now query any part of the sequence full sequence and check its secondary structure (via indices): NPXXY





In [ ]:
# Get angles from chosen starting point of the alignment ==> done via new column?

In [ ]:
def delta_phi_psi(phi_psi_1: list, phi_psi_2: list):
    # given two lists of phi and psi values calculate delta values (conformation change)
    return None

In [ ]:
def plot_ramachandran(phi_psi):
    pass

In [ ]:
def plot_delta_ramachandran(deltas):
    pass

In [ ]:
def plot_delta_per_gene(deltas, genes):
    pass

# Remark: PRECOG